## Imports

In [2]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpStatus, value #Linear Optimization Library

## Load Given Variables

**Note: I hid the output of some of these variables as they are quite long.**

#### N+1 cities (C0 ... Cn)

In [6]:
C = ['Allentown', 'Charlotte', 'Chicago', 'Dallas', 'Fontana', 'Nashville', 'Portland', 'Tucson']
C

['Allentown',
 'Charlotte',
 'Chicago',
 'Dallas',
 'Fontana',
 'Nashville',
 'Portland',
 'Tucson']

#### M+1 products (P0 ... Pn)

In [8]:
production_matrix = pd.read_excel("SBD2.xlsx", sheet_name="ProductionMatrix", header=1)
production_matrix = production_matrix.drop(['(blank)'], axis=1)
production_matrix = production_matrix.iloc[:-2]
production_matrix = production_matrix[["Row Labels", "Grand Total"]]
P = production_matrix["Row Labels"].tolist()
#P

#### Contribution Margin for Pm in Cn [Rm,n]

In [10]:
margin_matrix = pd.read_excel("SBD2.xlsx", sheet_name="MarginMatrix", header=1)
margin_matrix = margin_matrix.drop(['(blank)','Grand Total'], axis=1)
margin_matrix = margin_matrix.iloc[:-2]
margin_matrix = margin_matrix.fillna(0)
R = margin_matrix.drop(columns=["Row Labels"])
R

,Allentown,Charlotte,Chicago,Dallas,Fontana,Nashville,Portland,Tucson
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.417144,0.000000,0.229868
1,0.000000,0.248429,0.500158,0.172298,0.464887,0.000000,0.293694,0.000000
2,0.267870,0.312923,0.000000,0.000000,0.292407,0.000000,0.187039,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.409059
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.461297
...,...,...,...,...,...,...,...,...
110,0.102838,0.186494,0.957254,1.100048,0.000000,0.177778,0.403307,0.000000
111,0.324883,0.000000,0.000000,0.000000,0.000000,0.198844,0.000000,0.000000
112,0.000000,0.875111,0.713066,0.000000,0.660749,0.000000,0.260806,0.000000
113,0.369882,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.315296


#### Max Capacity in Cn [T0 to Tn]

In [12]:
city_cap_matrix = pd.read_excel("SBD2.xlsx", sheet_name="CityCapacity")
city_cap_matrix = city_cap_matrix.rename(columns={"Unnamed: 0": "Capacity Metric"})
city_cap_matrix = city_cap_matrix[city_cap_matrix["Capacity Metric"] == "Max Capacity"]
T = city_cap_matrix.drop(columns=["Capacity Metric"])
T = T.iloc[0].tolist()
T

[25000000,
 20000000,
 50000000,
 60000000,
 30000000,
 45000000,
 15000000,
 10000000]

#### Total Required Production of Pm in all cities [D0, ... Dm]

In [14]:
D = production_matrix["Grand Total"].tolist()
#D

In [15]:
#Used for Product Mix Constraint to set LB and UB for the new allocation matrix
prod_matrix = pd.read_excel("SBD2.xlsx", sheet_name="ProductionMatrix", header=1)
prod_matrix = prod_matrix.drop(['(blank)'], axis=1)
prod_matrix = prod_matrix.iloc[:-2]
prod_matrix = prod_matrix.drop(columns=["Row Labels", "Grand Total"])
prod_matrix = prod_matrix.fillna(0)

prod_matrix

,Allentown,Charlotte,Chicago,Dallas,Fontana,Nashville,Portland,Tucson
0,0.0,0.0,0.0,0.0,0.0,620932.0,0.0,213742.0
1,0.0,34146.0,453323.0,887734.0,586080.0,0.0,280893.0,0.0
2,687366.0,262349.0,0.0,0.0,324161.0,0.0,75844.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,254629.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,129325.0
...,...,...,...,...,...,...,...,...
110,565413.0,122624.0,1525662.0,1132476.0,0.0,1130398.0,175563.0,0.0
111,440054.0,0.0,0.0,0.0,0.0,172508.0,0.0,0.0
112,0.0,364832.0,818753.0,0.0,673321.0,0.0,71299.0,0.0
113,200687.0,0.0,0.0,0.0,0.0,0.0,0.0,247746.0


#### Calculate # of Cities and # of Products

In [17]:
N = len(C) #number of cities
M = len(P) #number of products

## Build the LP Optimization Model 

#### (A) Define Decision Variables


In [20]:
# Create decision variable matrix: A[m,n] = production allocation in A[product, city]
A = LpVariable.dicts(
    "Production",
    ((m, n) for m in range(0,M) for n in range(0,N)),
    lowBound=0, #non-negativity for dec. vars
    cat='Continuous'  # Use 'Integer' if units must be whole numbers
)

#### (B) Define Objective Function: Maximize Profit (Using Contribution Margin Here)

In [22]:
# Define Objective Function
model = LpProblem("OptimizeAllocation", LpMaximize) #Define the model

objective_expr = lpSum(A[m, n] * R.iloc[m, n] for m in range(0,M) for n in range(0,N)) #Maximizing CM
model += objective_expr, "Objective"


#### (C) Define Constraints

#### ① Non-negativity for decision variables. You cannot assign negative production allocation.
This is already in the Decision Variable definition, where lowBound = 0


#### ② In Shutdown City Cs , every Product  Px has zero allocation.


In [26]:
#2. If there is a shutdown city Cs, for every product Px: Ax,s = 0
S = C.index("Charlotte") #choose shutdown city

for m in range(0,M):
    model += A[m,S] == 0, f"Constraint4_{m}"

#### ③ Respect Product Capabilities: (For every City-Product pair Cn, Pm that doesn’t have the production capability, the new allocation is zero):


In [28]:
#3. For every combination Cn, Pm that is incompatible (Respect Product Capability)
for n in range(0,N):
    for m in range(0,M):
        if R.iloc[m,n] == 0:
            model += A[m,n] == 0, f"Constraint2_{m}_and_{n}"

#### ④ For every City, Cs , the Sum of City Production ≤ Total City Capacity:

In [30]:
#4. For every city, the SUM of the cities produciton <= total city capacity
for n in range(0,N):

    model += lpSum(A[m,n] for m in range(0,M)) <= T[n], f"Constraint1_{n}"


#### ⑤ For every Product, Px, the Sum of New Production of Product Must equal the Total Required Production of that Product (“Demand”).

In [32]:
#5. For every product, SUM of production of product in every city must equal the total requirement of product 
for m in range(0,M):
    model += lpSum(A[m,n] for n in range(0,N)) == D[m], f"Constraint3_{m}"

#### ⑥ Product Mix Constraint: (Must produce at least ¼ and max 3.5 times the product if produced that product last year).

In [34]:
#6. Add Constraints to ensure some product mix
for m in range(0,M):
    for n in range(0,N):
        if n == S: #cant force this constraint on a shutdown city
            continue
        model += A[m,n] >= 0.25 * prod_matrix.iloc[m,n], f"Constraint6_1_{m}_and_{n}"
        model += A[m,n] <= 3.5 * prod_matrix.iloc[m,n], f"Constraint6_2_{m}_and_{n}"

#### (D) Solve the Model

- If Optimal, passes "Capacity" Consideration.
- If Infeasible, does not pass "Capacity" Consideration given these assumptions, thus cannot consolidate that facility.

In [36]:
# Solve Model
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/r3/n8r88kl13ms0trd0_w4nm2kh0000gn/T/95a178686aec42cb9371b0898c8da595-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/r3/n8r88kl13ms0trd0_w4nm2kh0000gn/T/95a178686aec42cb9371b0898c8da595-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2418 COLUMNS
At line 6904 RHS
At line 9318 BOUNDS
At line 9319 ENDATA
Problem MODEL has 2413 rows, 920 columns and 4130 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 57 (-2356) rows, 237 (-683) columns and 474 (-3656) elements
0  Obj 42828252 Primal inf 87701967 (50) Dual inf 82.851232 (211)
37  Obj 1.1592739e+08 Primal inf 45817259 (36)
82  Obj 1.0813671e+08
Optimal - objective value 1.0813671e+08
After Postsolve, objective 1.0813671e+08, infeasibil

1

In [37]:
print("Status:", LpStatus[model.status])

Status: Optimal


#### (E) Convert Results to DF, then Output Optimal Allocation to Excel for Further Analysis

In [39]:
import pandas as pd

# Initialize list for (product, city, value)
data = []

for var in model.variables():
    if var.name.startswith("Production_") and var.varValue and var.varValue > 0:
        # Parse index from variable name
        stripped = var.name.replace("Production_(", "").replace(")", "")
        m_str, n_str = stripped.split(",_")
        m = int(m_str)
        n = int(n_str)

        # Map indices to names
        product_name = P[m]
        city_name = C[n]
        value = var.varValue

        data.append((product_name, city_name, value))

# Instantiate DF
df = pd.DataFrame(data, columns=["Product", "City", "Production"])

# Pivot to Excel format
matrix_df = df.pivot(index="Product", columns="City", values="Production").fillna(0)
matrix_df = matrix_df.round(2)
matrix_df


City,Allentown,Chicago,Dallas,Fontana,Nashville,Portland,Tucson
Product,,,,,,,
Aerosol Bottle,0.00,0.00,0.0,0.00,781238.5,0.00,53435.5
Aerosol Can,0.00,1586630.50,221933.5,363388.75,0.0,70223.25,0.0
Bottle with Cap,1249718.80,0.00,0.0,81040.25,0.0,18961.00,0.0
Bottle with Logo,0.00,0.00,0.0,0.00,0.0,0.00,254629.0
Branded Bottle,0.00,0.00,0.0,0.00,0.0,0.00,129325.0
...,...,...,...,...,...,...,...
Vacuum-Insulated Bottle,141353.25,381415.50,3802877.0,0.00,282599.5,43890.75,0.0
Wide-Mouth Bottle,569435.00,0.00,0.0,0.00,43127.0,0.00,0.0
Wide-Mouth Can,0.00,1742050.00,0.0,168330.25,0.0,17824.75,0.0


In [40]:
#Export to Excel 
#Outputs Match to "2. Capacity + 3. Financial" Tab in Excel
matrix_df.to_excel("production_output11.xlsx", sheet_name="Production Plan")